In [1]:
#! /usr/bin/env python3
%matplotlib widget
import rospy
import sys
import select
import actionlib
import actionlib.msg
import assignment_2_2022.msg

from assignment_2_2022.srv import target_srv
from nav_msgs.msg import Odometry
from assignment_2_2022.msg import PoseVelocity, PlanningAction, PlanningGoal
from geometry_msgs.msg import Point, Pose, Twist 
from threading import Thread
from std_msgs.msg import String

import matplotlib.pyplot as plt
import tf
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation    

In [2]:
class Visualiser:
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.x_data, self.y_data = [] , []

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln

    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
    
    def display_graph(self):
        plt.plot(self.x_data, self.y_data)
        plt.show()
        


In [ ]:
class RobotController:
    
    def __init__(self):
        rospy.init_node('robot_controller') # initialize node
        self.vis = Visualiser()
        self.pub = rospy.Publisher("/PoseVelocity", PoseVelocity, queue_size=10)
        rospy.Subscriber("/odom", Odometry, self.vis.odom_callback)  # add odom subscriber here

    def call_srv(self):
        rospy.wait_for_service('srv')
        srv = rospy.ServiceProxy('srv', target_srv)
        resp = srv()
        print("Number of goals cancelled", resp.cancelled)
        print("Number of goals reached", resp.reached)

    def pub_data(self, msg):
        PoseVelocity_output = PoseVelocity()
        position = msg.pose.pose.position
        velocity = msg.twist.twist.linear

        PoseVelocity_output_px = position.x
        PoseVelocity_output_py = position.y
        PoseVelocity_output_vx = velocity.x
        PoseVelocity_output_vy = velocity.y

        self.pub.publish(PoseVelocity_output)

    def action_client(self):
        action_client = actionlib.SimpleActionClient("/reaching_goal", PlanningAction)
        action_client.wait_for_server()

        while not rospy.is_shutdown():
            print("Enter the coordinates x and y")
            x = float(input("Enter a goal x: "))
            y = float(input("Enter a goal y: "))

            goal = PlanningGoal()
            goal.target_pose.pose.position.x = x
            goal.target_pose.pose.position.y = y

            self.vis.target_x = x
            self.vis.target_y = y

            action_client.send_goal(goal)

            # Wait for the goal to complete or be cancelled
            action_client.wait_for_result(rospy.Duration(0.1))

            # Check if the goal was cancelled
            if action_client.get_state() == actionlib.GoalStatus.PREEMPTED:
                print("The goal has been cancelled")
                self.vis.target_x = None
                self.vis.target_y = None

            # Break out of the loop if the goal was successful
            if action_client.get_state() == actionlib.GoalStatus.SUCCEEDED:
                break


    def cancel_target(self):
        print("Press x to cancel the target")
        input = select.select([sys.stdin], [], [], 3)[0]

        if input:
            i = sys.stdin.readline().rstrip()
            if i == "x":
                self.action_client.cancel_goal()
                print("The goal has been cancelled")
                self.vis.target_x = None
                self.vis.target_y = None

    def main(self):
        self.action_client()


if __name__ == '__main__':
    rc = RobotController()
    rc.main() 

Enter the coordinates x and y
Enter a goal x: 1
Enter a goal y: 2
